In [1]:
import folium
import requests
import numpy as np
import pandas as pd

In [3]:
accept = pd.read_csv("accepted_2007_to_2018Q4.csv", low_memory=False)

In [4]:
raw_data = accept[(accept.loan_status == "Charged Off") & (accept.application_type == "Individual") | (accept.loan_status == "Fully Paid") & (accept.application_type == "Individual")]

In [5]:
copy_data = raw_data.copy()
copy_data["dti"] = copy_data.dti.apply(lambda x: 0 if x <= 0 else x if x <= 40 else 40)
copy_data["revol_util"] = copy_data.revol_util.apply(lambda x: 100 if x > 100 else x)
copy_data["last_fico_range_high"] = np.where(copy_data.last_fico_range_low == 0, 304, copy_data.last_fico_range_high)
copy_data["last_fico_range_low"] = np.where(copy_data.last_fico_range_low == 0, 300, copy_data.last_fico_range_low)

In [6]:
charged_off_data = (copy_data[["addr_state", "loan_status"]].groupby("addr_state").loan_status.value_counts(normalize=True).loc[pd.IndexSlice[:, "Charged Off"]] * 100).reset_index()
last_fico_score_data = copy_data[["addr_state", "last_fico_range_low", "last_fico_range_high"]].groupby("addr_state").mean().mean(axis=1).reset_index().set_axis(["addr_state", "last_fico_score"], axis=1)
dti_data = copy_data[["addr_state", "dti"]].groupby("addr_state").mean().reset_index()
int_rate_data = copy_data[["addr_state", "int_rate"]].groupby("addr_state").int_rate.mean().reset_index()

In [7]:
state_geo = requests.get("https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json").json() # US states
state_geo.get("features").append({"type": "Feature", 
                                  "id": "DC", 
                                  "properties": {"name": "District of Columbia"},
                                  "geometry": {"type": "Polygon",
                                               "coordinates": [[[-77.035264,38.993869], 
                                                                [-77.117418,38.933623], 
                                                                [-77.040741,38.791222], 
                                                                [-76.909294,38.895284],
                                                                [-77.035264,38.993869]]]}})

m = folium.Map(location=[48, -102], tiles=folium.TileLayer("cartodbpositron", name="base_map"), zoom_start=3, zoom_control=False)

charged_off_chor = folium.Choropleth(geo_data=state_geo, 
                                     data=charged_off_data, 
                                     columns=charged_off_data.columns, 
                                     key_on="feature.id", 
                                     fill_color="OrRd", 
                                     line_weight=0.7, 
                                     line_opacity=0.4, 
                                     name="Charged Off 비율", 
                                     legend_name="Charged Off 비율", 
                                     show=False)

for data in charged_off_chor.geojson.data.get("features"):
    data.get("properties")["id"] = f"{data.get('properties').get('name')} ({data.get('id')})"
    data.get("properties")["charged_off_rate"] = charged_off_data[charged_off_data.addr_state == data.get("id")]["proportion"].values.round(2)[0]
folium.GeoJsonTooltip(fields=["id", "charged_off_rate"], aliases=["주 이름", "Charged Off 비율"], style="font-size: 12px").add_to(charged_off_chor.geojson)


last_fico_score_chor = folium.Choropleth(geo_data=state_geo, 
                                         data=last_fico_score_data, 
                                         columns=last_fico_score_data.columns, 
                                         key_on="feature.id", 
                                         fill_color="OrRd", 
                                         line_weight=0.7, 
                                         line_opacity=0.4, 
                                         name="평균 최근 신용점수", 
                                         legend_name="평균 최근 신용점수", 
                                         show=False)

for data in last_fico_score_chor.geojson.data.get("features"):
    data.get("properties")["id"] = f"{data.get('properties').get('name')} ({data.get('id')})"
    data.get("properties")["last_fico_score"] = last_fico_score_data[last_fico_score_data.addr_state == data.get("id")]["last_fico_score"].values.round()[0]
folium.GeoJsonTooltip(fields=["id", "last_fico_score"], aliases=["주 이름", "평균 최근 신용점수"], style="font-size: 12px").add_to(last_fico_score_chor.geojson)


dti_chor = folium.Choropleth(geo_data=state_geo, 
                             data=dti_data, 
                             columns=dti_data.columns, 
                             key_on="feature.id", 
                             fill_color="OrRd", 
                             line_weight=0.7, 
                             line_opacity=0.4, 
                             name="평균 dti", 
                             legend_name="평균 dti", 
                             show=False)

for data in dti_chor.geojson.data.get("features"):
    data.get("properties")["id"] = f"{data.get('properties').get('name')} ({data.get('id')})"
    data.get("properties")["dti"] = dti_data[dti_data.addr_state == data.get("id")]["dti"].values.round(2)[0]
folium.GeoJsonTooltip(fields=["id", "dti"], aliases=["주 이름", "평균 dti"], style="font-size: 12px").add_to(dti_chor.geojson)


int_rate_chor = folium.Choropleth(geo_data=state_geo, 
                                  data=int_rate_data, 
                                  columns=int_rate_data.columns, 
                                  key_on="feature.id", 
                                  fill_color="OrRd", 
                                  line_weight=0.7, 
                                  line_opacity=0.4, 
                                  name="평균 이자율", 
                                  legend_name="평균 이자율", 
                                  show=False)

for data in int_rate_chor.geojson.data.get("features"):
    data.get("properties")["id"] = f"{data.get('properties').get('name')} ({data.get('id')})"
    data.get("properties")["int_rate"] = int_rate_data[int_rate_data.addr_state == data.get("id")]["int_rate"].values.round(2)[0]
folium.GeoJsonTooltip(fields=["id", "int_rate"], aliases=["주 이름", "평균 이자율"], style="font-size: 12px").add_to(int_rate_chor.geojson)

charged_off_chor.add_to(m)
last_fico_score_chor.add_to(m)
dti_chor.add_to(m)
int_rate_chor.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m